In [1]:
pip install -U langchain-community tavily-python langchainhub langgraph


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.3 MB 3.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.3 MB 8.3 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 10.4 MB/s eta 0:00:01
   --------------------------------- ------ 2.0/2.3 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/91.1 kB ? eta -:--:--
   ---------------------------------------- 91.1/91.1 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ----------------------- ---------------- 0.6/1.0 MB 18.8 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 12.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/396.4 kB ? eta -:--:--
   --------------------------------------- 396.4/396.4 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: l

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 2)

search.invoke("What is the weather in Herlev in Denmark today?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Herlev', 'region': 'Hovedstaden', 'country': 'Denmark', 'lat': 55.72, 'lon': 12.45, 'tz_id': 'Europe/Copenhagen', 'localtime_epoch': 1725474332, 'localtime': '2024-09-04 20:25'}, 'current': {'last_updated_epoch': 1725473700, 'last_updated': '2024-09-04 20:15', 'temp_c': 24.1, 'temp_f': 75.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 80, 'wind_dir': 'E', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 61, 'cloud': 0, 'feelslike_c': 25.5, 'feelslike_f': 78.0, 'windchill_c': 21.6, 'windchill_f': 70.9, 'heatindex_c': 24.5, 'heatindex_f': 76.1, 'dewpoint_c': 13.6, 'dewpoint_f': 56.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 26.9, 'gust_kph': 43.3}}"},
 {'url': 'https://www.weathertab.com/en/d/e/09/kingdom-of-denmark/region-hoveds

In [3]:
#List of tools
tools = [search]

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [9]:
# from langchain_core.messages import HumanMessage

# response = model.invoke([HumanMessage(content="Hi")])
# response.content

'Hello! How can I assist you today?'

In [4]:
model_with_tools = model.bind_tools(tools)

In [13]:
# model_with_tools.invoke([HumanMessage(content="Hi")])

# print(f"ContentString: {response.content}")
# print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [7]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="How is the weather in Bangalore?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Bangalore'}, 'id': 'call_Q4Apvpy4vzV3n9NLlDvgFlbO', 'type': 'tool_call'}]


In [11]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(model,tools)

In [24]:
response = agent_executor.invoke({"messages": [HumanMessage(content="How is the weather in Bangalore?")]})
response['messages']

[HumanMessage(content='How is the weather in Bangalore?', id='0d1588b1-21a3-4d08-b2d0-12bed39818da'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LSo5agDIkeSxFNQIXsyM7IyP', 'function': {'arguments': '{\n  "query": "current weather in Bangalore"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-243e1f57-a256-4e32-89e2-35b2cbfce5ce-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Bangalore'}, 'id': 'call_LSo5agDIkeSxFNQIXsyM7IyP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 22, 'total_tokens': 110}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Bangalore\', \'region\': \'Karnataka\', \